# 크롤러 사용할 때 기억해야 하는 것 


> - 크롤러 분류하기
- 크롤러 만들때 주의해야 하는 것
- 여러 번 사용을 전제로 설계하기
- 크롤링 대상의 변화에 대응하기

## 크롤러 분류하기 
> 3가지 분류기준

> - 상태를 가지고 있는지 : Statefull, Stateless
- 자바스크립트를 실행할 수 있는지
- 불특정 다수의 사이트를 대상으로 하는지

cf. Selenium : 브라우저를 자동으로 조작할 수 있게 해주는 도구

## 크롤러 만들때 주의해야 하는 것 
> - robots.txt : Robots Exclusion Protocol 
- \- 웹사이트 최상위 디렉토리에 배치 
- \- 디렉티브 : User-agent, Disallow, Allow, Sitemap, Crawl-delay

robots.txt 파싱
> RobotFileParser : robots.txt를 파싱하기 위한 urllib.robotparser

In [5]:
import urllib.robotparser

rp = urllib.robotparser.RobotFileParser()

In [18]:
# url = 'http://wikibook.co.kr'
# url = 'https://www.google.com'
# url = 'http://www.naver.com'
# url ='http://www.youtube.com'
url = 'http://www.hani.co.kr/'

rp_url = url + '/robots.txt'
print(rp_url)

rp.set_url(rp_url)
rp.read()

http://www.hani.co.kr//robots.txt


cf> robots.txt 주소로 연결해 갔을 때 뜨는 user-agent 목록을 보면 allow, disallow로 허가된 것과 안되는 것을 구분해 놓았다. $표시가 있는 것은 돈 내면 가능하다는 것.

In [19]:
# 해당 URL을 크롤링해도 괜찮은지 확인
rp.can_fetch('mybot', url)

False

**c4-07_error_handling **
> 상태코드에 맞는 오류 처리

> - 404번 문제가 가장 많이 발생한다.(보통은 오타)
- 500번 에러가 가장 심각하다.(서버 내부 문제임)

In [20]:
import time
import requests
# 일시적인 오류를 나타내는 상태 코드를 지정합니다.
TEMPORARY_ERROR_CODES = (408, 500, 502, 503, 504)  

def main():
    """
    메인 처리입니다.
    """
    response = fetch('http://httpbin.org/status/200,404,503')
    if 200 <= response.status_code < 300:
        print('Success!')
    else:
        print('Error!')

def fetch(url):
    """
    지정한 URL에 요청한 뒤 Response 객체를 반환합니다.
    일시적인 오류가 발생하면 최대 3번 재시도합니다.
    """
    max_retries = 3  # 최대 3번 재시도합니다. (오류가 나도 최대 3번까지 시도해보는 것)
    retries = 0      # 현재 재시도 횟수를 나타내는 변수입니다.
    
    while True:
        try:
            print('Retrieving {0}...'.format(url))
            response = requests.get(url)
            print('Status: {0}'.format(response.status_code))
            if response.status_code not in TEMPORARY_ERROR_CODES:
                return response  # 일시적인 오류가 아니라면 response를 반환합니다.
            
        except requests.exceptions.RequestException as ex:
            # 네트워크 레벨 오류(RequestException)의 경우 재시도합니다.
            print('Exception occured: {0}'.format(ex))
            retries += 1
            if retries >= max_retries:
                # 재시도 횟수 상한을 넘으면 예외를 발생시켜버립니다.
                raise Exception('Too many retries.')  
            # 지수 함수적으로 재시도 간격을 증가합니다(**는 제곱 연산자입니다).
            wait = 2**(retries - 1)  
            print('Waiting {0} seconds...'.format(wait))
            time.sleep(wait)  # 대기합니다.

if __name__ == '__main__':
    main()
    

Retrieving http://httpbin.org/status/200,404,503...
Status: 200
Success!


In [ ]:
# cf> 개발 쪽 생각 있으면, try exception 구문은 필수다.
# 로직을 써야 하는 부분은 try 부분에 집어넣으면 좋다.
# 프로그램도 안정적이고 가독성도 높아지는 장점이 있다.

In [21]:
url = 'http://www.google.com'
fetch(url)

Retrieving http://www.google.com...
Status: 200


<Response [200]>

In [22]:
url = 'http://www.naver.com'
fetch(url)

Retrieving http://www.naver.com...
Status: 200


<Response [200]>

In [23]:
url = 'http://www.nagooglever.com'
fetch(url)

Retrieving http://www.nagooglever.com...
Status: 200


<Response [200]>

In [24]:
url = 'http://www.notexist.kr'
fetch(url)

# 오류 발생 시 나오는 < Exception : Too many retries > 문구는 직접 만들어낸 오류 문구이다.

Retrieving http://www.notexist.kr...
Exception occured: HTTPConnectionPool(host='www.notexist.kr', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000000064C14A8>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',))
Waiting 1 seconds...
Retrieving http://www.notexist.kr...
Exception occured: HTTPConnectionPool(host='www.notexist.kr', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000000064C1518>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',))
Waiting 2 seconds...
Retrieving http://www.notexist.kr...
Exception occured: HTTPConnectionPool(host='www.notexist.kr', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000000064C1278>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',))


Exception: Too many retries.

**c4-08_error_handling_with_retrying** 
> retrying을 이용한 재시도 처리

In [27]:
! pip install retrying

  Running setup.py bdist_wheel for retrying: started
  Running setup.py bdist_wheel for retrying: finished with status 'done'
  Stored in directory: C:\Users\student\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [28]:
import requests
from retrying import retry  
import time

# 일시적인 오류를 나타내는 상태 코드를 지정합니다.
TEMPORARY_ERROR_CODES = (408, 500, 502, 503, 504)  

def main():
    """
    메인 처리입니다.
    """
    response = fetch('http://httpbin.org/status/200,404,503')
    if 200 <= response.status_code < 300:
        print('Success!')
    else:
        print('Error!')

        
# decorator 
# stop_max_attempt_number로 최대 재시도 횟수를 지정합니다.
# wait_exponential_multiplier로 특정한 시간 만큼 대기하고 재시도하게 합니다. 단위는 밀리초로 입력합니다.
@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000)
# 함수나 클래스 바깥에 필요한 정보들을 가져올 수 있도록 지정해주는 것이 @를 활용한 decorator 기능이다.
def fetch(url):
    """
    지정한 URL에 접근한 뒤 Response 객체를 반환합니다.
    일시적인 오류가 발생할 경우 3번까지 재시도합니다.
    """
    print('Retrieving {0}...'.format(url))
    response = requests.get(url)
    print('Status: {0}'.format(response.status_code))
    if response.status_code not in TEMPORARY_ERROR_CODES:
        # 오류가 없다면 response를 반환합니다.
        return response
    # 오류가 있다면 예외를 발생시킵니다.
    raise Exception('Temporary Error: {0}'.format(response.status_code))

if __name__ == '__main__':
    main()

Retrieving http://httpbin.org/status/200,404,503...
Status: 200
Success!


### 데코레이터 기능

In [29]:
# decorator.py
# -*- coding: utf-8 -*-
def decorator_function(original_function):
    def wrapper_function():
        print('{} 함수가 호출되기전 입니다.'.format(original_function.__name__))
        return original_function()
    return wrapper_function


@decorator_function  #1
def display_1():
    print('display_1 함수가 실행됐습니다.')


@decorator_function  #2
def display_2():
    print('display_2 함수가 실행됐습니다.')

# display_1 = decorator_function(display_1)  #3
# display_2 = decorator_function(display_2)  #4

display_1()
print('-'*50)
display_2()

display_1 함수가 호출되기전 입니다.
display_1 함수가 실행됐습니다.
--------------------------------------------------
display_2 함수가 호출되기전 입니다.
display_2 함수가 실행됐습니다.


In [30]:
# @decorator_function 데코레이터를 입력함으로써,
# 원래라면 display_1과 display_2가 실행되어야 하지만,
# decorator_function 이 먼저 실행되는 것을 확인할 수 있다.

## 여러 번 사용을 전제로 설계하기 
> 크롤링 도중에 오류가 발생했을때를 위한 처리

> - 이후에 변경된 데이터를 추가로 추출할 수 있게 하기 위해
- 오류 등으로 중간에 중지되었을 때, 중간부터 다시 재개하기 위해

**c4-09_request_with_cache**
> CacheControl을 사용해 캐시 처리

In [35]:
! pip install CacheControl

  Running setup.py bdist_wheel for CacheControl: started
  Running setup.py bdist_wheel for CacheControl: finished with status 'done'
  Stored in directory: C:\Users\student\AppData\Local\pip\Cache\wheels\36\bd\5f\dbbee4f2d51f97ecd12a363f870361179cb1fd4bc1174ea08a
Successfully built CacheControl


In [36]:
import requests
from cachecontrol import CacheControl  

session = requests.session()

# session을 래핑한 cached_session 만들기
cached_session = CacheControl(session)

url = 'https://docs.python.org/3/'
# 첫 번째는 캐시돼 있지 않으므로 서버에서 추출한 이후 캐시합니다.
response = cached_session.get(url)
print(response.from_cache)

# 두 번째는 ETag와 Last-Modified 값을 사용해 업데이트됐는지 확인합니다.
# 변경사항이 없는 경우에는 콘텐츠를 캐시에서 추출해서 사용하므로 빠른 처리가 가능합니다.
response = cached_session.get(url)
print(response.from_cache) 

False
True


In [37]:
response = cached_session.get(url)
print(response.from_cache) 

True


cf> 캐쉬에 저장된 정보를 가지고서 빠른 처리가 가능하도록 하는데, 한번 캐시해 놓으면 다음부터는 해당 캐시로 실행되게 된다.(?)

## 크롤링 대상의 변화에 대응하기 
> - 변화 감지하기
- 변화 통지하기

### 변화 감지 
**c4-10_validate_with_re**

In [43]:
import re

value = '3,000'
value = '3천'

# 숫자와 쉼표만을 포함한 정규 표현식에 매치하는지 확인합니다.
if not re.search(r'^[0-9,]+$', value):
    # 정규표현식 앞에 붙은 r은 rawdata를 의미한다. 만약 b라면 bytedata를 의미
    # ^표시가 정규표현식 바깥에 존재한다면 그것은 내가 찾는 문장의 첫번째 위치를 의미하며,
    # $ 표시는 끝을 의미한다.
    # 값이 제대로 돼 있지 않다면 예외를 발생시킵니다.
    raise ValueError('Invalid price')

ValueError: Invalid price

In [44]:
# ! pip install voluptuous

**c4-11_validate_with_voluptuous **

In [45]:
from voluptuous import Schema, Match  

# 다음 4개의 규칙을 가진 스키마를 정의합니다
schema = Schema({                  # 규칙1: 객체는 dict 자료형
    'name' : str,                  # 규칙2：name은 str(문자열) 자료형
    'price': Match(r'^[0-9,]+$'),  # 규칙3：price가 정규 표현식에 맞는지 확인
}, required=True)                  # 규칙4：dict의 키는 필수

In [46]:
# Schema 객체는 함수처럼 호출해서 사용합니다.
# 매개변수에 대상을 넣으면 유효성 검사를 수행합니다.
schema({
    'name' : '포도',
    'price': '3,000',
})  # 유효성 검사를 통과하므로 아무 문제 없음

{'name': '포도', 'price': '3,000'}

In [50]:
# 유효성 검사를 통과하지 못 하므로, MultipleInvalid 예외가 발생
schema({
    'name' : None,
    'price': '3,000',
})  
# 규칙2번을 위반하였기 때문에 오류가 뜬다.

MultipleInvalid: expected str for dictionary value @ data['name']

In [52]:
# 유효성 검사를 통과하지 못 하므로, MultipleInvalid 예외가 발생
schema({
    'name' : '오렌지',
    'price': '3천',
})  
# 규칙 3번을 위반하였으므로 오류가 뜬다.

MultipleInvalid: does not match regular expression for dictionary value @ data['price']

In [53]:
# 미국의 경우에는 개발과 테스트 기간이 5:5의 비율이다.
# 보통은 테스트를 통해서 품질개선을 한다고 생각하지만,
# 실제로는 테스트를 통해서는 결함을 찾는 것까지의 역할이고,
# 그것을 토대로 다시 품질을 개선하는 것은 개발자의 몫이다.

## 변화 통지 
**c4-12_send_email **